In [68]:
import sqlite3
import pandas as pd

In [69]:
'''
conn = sqlite3.connect('SDOH.db')
main_table = 'Main'
df = pd.read_sql_table('Main', conn)
'''

"\nconn = sqlite3.connect('SDOH.db')\nmain_table = 'Main'\ndf = pd.read_sql_table('Main', conn)\n"

In [70]:
conn = sqlite3.connect('SDOH.db')
main_table = 'Main'
main_query = f'SELECT * FROM {main_table}'

In [71]:
df = pd.read_sql_query(main_query, conn)

In [72]:
df.head(3)

,ID,RECORDED_TIME,GENDER,AGE,RACE_NAME,ETHNICITY,ENC_TYPE_NAME,DEPARTMENT_ID,CAD,CHF,COPD,DIABETES,HYPERTENSION,RESULTS,QUESTION_ID,QUESTION_NAME,DOCUMENTATION_SOURCE_NAME,EMPY_STATUS_NAME,EDU_LEVEL_NAME,TOBACCO_USER_NAME
0,4430,01:13.0,Female,74,White,Non-Hispanic,History,601,0,0,0,0,0,1,1572879820,R FOOD INSECURITY WORRY,MyChart (Questionnaires),Not Employed,NA,Never
1,103536,46:00.0,Female,34,White,Non-Hispanic,Hospital Encounter,NA,0,0,0,0,0,4,1572879810,R FINANCIAL RESOURCE STRAIN BASICS,Flowsheet Web Popup,NA,NA,Quit
2,125032,21:00.0,Female,33,Black / African American,Non-Hispanic,Psych Confidential,2408,0,0,0,0,1,None of the above,8258,MH R SDOH DO YOU HAVE PROBLEMS WITH ANY OF THE...,MyChart (Questionnaires),Not Employed,"Some college, no degree",Never


In [73]:
results_type_counts = df['RESULTS'].apply(type).value_counts()
question_id_type_counts = df['QUESTION_ID'].apply(type).value_counts()
print(f'Data type counts for RESULTS column:\n{results_type_counts}\n'
      f'Data type counts for QUESTION_ID column:\n{question_id_type_counts}')

Data type counts for RESULTS column:
<class 'str'>    800000
Name: RESULTS, dtype: int64
Data type counts for QUESTION_ID column:
<class 'str'>    800000
Name: QUESTION_ID, dtype: int64


In [74]:
df['QUESTION_ID'] = df['QUESTION_ID'].astype(int)

In [75]:
# For question 1570008806 the RESULTS column should be directly mapped to the new answer column, but leave blank if RESULT = 'NA'
# Leaving out question 8258 due to difficulty creating the map coding and does not seem like a necessary question to keep
# For question 14208 the RESULTS are similar to how they are in 8258 where multiple answers are able to be chosen and they are separated by 
#   a semicolon. I am thinking that we could create a new table with the first column as ID and then the other 12 columns represent the 
#   resource that they are asking for assistance with, where the value is either 1 or 0.

value_mapping = {
    (1572879820, '1'): 'Never',
    (1572879820, '2'): 'Sometimes',
    (1572879820, '3'): 'Often',

    (1572879821, '1'): 'Never',
    (1572879821, '2'): 'Sometimes',
    (1572879821, '3'): 'Often',

    (1572879817, '1'): 'Yes',
    (1572879817, '2'): 'No',

    (1572879818, '1'): 'Yes',
    (1572879818, '2'): 'No',

    (8261, 'Yes'): 'Yes',
    (8261, 'No'): 'No',

    (8306, 'On a cell phone'): 'On a cell phone',
    (8306, 'At work'): 'At work',
    (8306, 'Other location (such as a library)'): 'Other location (such as a library)',
    (8306, 'Multiple access to the internet'): 'Multiple access to the internet',
    (8306, 'I do not have access to the internet'): 'I do not have access to the internet',

    (1570008805, '1'): 'Yes',
    (1570008805, '2'): 'No',

    (1570008807, '1'): 'Yes',
    (1570008807, '2'): 'No',

    (8307, 'Yes'): 'Yes',
    (8307, 'No'): 'No',
    (8307, 'Already shut off'): 'Already shut off',

    (1572879813, '1'): 'Yes',
    (1572879813, '2'): 'No',

    (1572879812, '1'): 'Yes',
    (1572879812, '2'): 'No',

    (1572879815, '1'): 'Yes',
    (1572879815, '2'): 'No',

    (1572879814, '1'): 'Yes',
    (1572879814, '2'): 'No',

    (1572879810, '1'): 'Very hard',
    (1572879810, '2'): 'Hard',
    (1572879810, '3'): 'Somewhat hard',
    (1572879810, '4'): 'Not very hard',
    (1572879810, '5'): 'Not hard at all',

    (1572879823, '1'): 'Not at all',
    (1572879823, '2'): 'Only a little',
    (1572879823, '3'): 'To some extent',
    (1572879823, '4'): 'Rather much',
    (1572879823, '5'): 'Very Much',

    (1572879825, '1'): 'Never',
    (1572879825, '2'): 'Once a week',
    (1572879825, '3'): 'Twice a week',
    (1572879825, '4'): 'Three times a week',
    (1572879825, '5'): 'More than 3 x week',

    (1572879826, '1'): 'Never',
    (1572879826, '2'): 'Once a week',
    (1572879826, '3'): 'Twice a week',
    (1572879826, '4'): 'Three times a week',
    (1572879826, '5'): 'More than 3 x week',

    (1572879827, '1'): 'Never',
    (1572879827, '2'): '1 to 4 times per year',
    (1572879827, '3'): 'more than 4 times per year',

    (1572879828, '1'): 'Yes',
    (1572879828, '2'): 'No',

    (1572879829, '1'): 'Never',
    (1572879829, '2'): '1 to 4 times per year',
    (1572879829, '3'): 'more than 4 times per year',

    (1572879830, '3'): 'Married',
    (1572879830, '4'): 'Widowed',
    (1572879830, '5'): 'Divorced',
    (1572879830, '6'): 'Separated',
    (1572879830, '7'): 'Never married',
    (1572879830, '8'): 'Living with partner',

    (1572879832, '0'): int(0),
    (1572879832, '1'): 1,
    (1572879832, '2'): 2,
    (1572879832, '3'): 3,
    (1572879832, '4'): 4,
    (1572879832, '5'): 5,
    (1572879832, '6'): 6,
    (1572879832, '7'): 7,

    (1572879833, '0'): int(0),
    (1572879833, '1'): 1,
    (1572879833, '2'): 2,
    (1572879833, '3'): 3,
    (1572879833, '4'): 4,
    (1572879833, '5'): 5,
    (1572879833, '6'): 6,
    (1572879833, '7'): 7,
    (1572879833, '8'): 8,
    (1572879833, '9'): 9,
    (1572879833, '10'): 10,
    (1572879833, '11'): 11,
    (1572879833, '12'): 12,
    (1572879833, '13'): 13,
    (1572879833, '14'): 14,
    (1572879833, '15'): 15,

    (8260, 'I do not have a steady place to live (I am temporarily staying with others, in a hotel, in a shelter, living outside on the street, on a beach, in a car, abandoned building, bus or train station, or in a park)'): 'I do not have a steady place to live (I am temporarily staying with others, in a hotel, in a shelter, living outside on the street, on a beach, in a car, abandoned building, bus or train station, or in a park)',
    (8260, 'I have a place to live today, but I am worried about losing it in the future'): 'I have a place to live today, but I am worried about losing it in the future',
    (8260, 'I have a steady place to live'): 'I have a steady place to live',

    (13895, 'Yes'): 'Yes',
    (13895, 'No'): 'No'
}

In [77]:
# Separate tables were created for 8258 and 14208 with binary encodings for each column where each column represents a possible answer. In
#   the RESULTS_NAME column we still need a value representation for these QUESTION_ID's. The value will simply mean that there is another
#   table created for that QUESTION_ID that can be joined to this main table.
# I am choosing the value representation to be 'HousingProblems table' for 8258 and 'RequestForAssistance table' for 14208

value_mapping2 = {
    (8258): 'HousingProblems table',
    (14208): 'RequestForAssistance table',
}

In [78]:
df['RESULTS_NAME'] = df.apply(lambda row: value_mapping.get((row['QUESTION_ID'], row['RESULTS']), None)
                              or value_mapping2.get(row['QUESTION_ID'], None)
                              or (row['RESULTS'] if row['QUESTION_ID'] == 1570008806 and row['RESULTS'] != 'NA' else None), axis = 1)

In [79]:
df.head(10)

,ID,RECORDED_TIME,GENDER,AGE,RACE_NAME,ETHNICITY,ENC_TYPE_NAME,DEPARTMENT_ID,CAD,CHF,...,DIABETES,HYPERTENSION,RESULTS,QUESTION_ID,QUESTION_NAME,DOCUMENTATION_SOURCE_NAME,EMPY_STATUS_NAME,EDU_LEVEL_NAME,TOBACCO_USER_NAME,RESULTS_NAME
0,4430,01:13.0,Female,74,White,Non-Hispanic,History,601,0,0,...,0,0,1,1572879820,R FOOD INSECURITY WORRY,MyChart (Questionnaires),Not Employed,NA,Never,Never
1,103536,46:00.0,Female,34,White,Non-Hispanic,Hospital Encounter,NA,0,0,...,0,0,4,1572879810,R FINANCIAL RESOURCE STRAIN BASICS,Flowsheet Web Popup,NA,NA,Quit,Not very hard
2,125032,21:00.0,Female,33,Black / African American,Non-Hispanic,Psych Confidential,2408,0,0,...,0,1,None of the above,8258,MH R SDOH DO YOU HAVE PROBLEMS WITH ANY OF THE...,MyChart (Questionnaires),Not Employed,"Some college, no degree",Never,HousingProblems table
3,110288,48:00.0,Male,47,White,Non-Hispanic,Office Visit,1140,0,0,...,0,0,1,1572879828,R SOCIAL CONNECTIONS CLUB MEMBERSHIP,MyChart (Questionnaires),Full Time,NA,Never,Yes
4,171371,23:00.0,Female,62,White,Non-Hispanic,Office Visit,721,0,0,...,1,1,Multiple access to the internet,8306,MH R DO YOU HAVE ACCESS TO THE INTERNET AT ANY...,MyChart (Questionnaires),Full Time,"Bachelor's degree (e.g., BA, AB, BS)",Quit,Multiple access to the internet
5,130830,48:00.0,Male,67,White,Unavailable,Nurse Visit,1140,0,0,...,0,0,5,1572879810,R FINANCIAL RESOURCE STRAIN BASICS,MyChart (Questionnaires),Full Time,NA,Never,Not hard at all
6,21492,16:00.0,Male,57,Black / African American,Non-Hispanic,Office Visit,474,0,0,...,1,1,1,1572879829,R SOCIAL CONNECTIONS CLUB ATTENDANCE,MyChart (Questionnaires),Full Time,NA,Never,Never
7,55959,22:00.0,Female,36,White,Non-Hispanic,Office Visit,2381,0,0,...,0,0,2,1572879814,R INTIMATE PARTNER VIOLENCE RAPE HARK,MyChart (Questionnaires),Full Time,NA,Never,No
8,138891,19:00.0,Female,27,Declined To Answer,Declined To Answer,Office Visit,1160,0,0,...,0,0,98,1572879826,R SOCIAL CONNECTIONS GET TOGETHER,MyChart (Questionnaires),NA,12th grade,Quit,None
9,10201,39:34.0,Female,67,White,Non-Hispanic,History,601,0,0,...,0,0,3,1572879830,R SOCIAL CONNECTIONS MARITAL STATUS,MyChart (Questionnaires),Full Time,NA,Quit,Married


In [83]:
df[(df['QUESTION_ID'] == 1572879832) & (df['RESULTS'] == '0')]

,ID,RECORDED_TIME,GENDER,AGE,RACE_NAME,ETHNICITY,ENC_TYPE_NAME,DEPARTMENT_ID,CAD,CHF,...,DIABETES,HYPERTENSION,RESULTS,QUESTION_ID,QUESTION_NAME,DOCUMENTATION_SOURCE_NAME,EMPY_STATUS_NAME,EDU_LEVEL_NAME,TOBACCO_USER_NAME,RESULTS_NAME
68,68065,05:05.0,Female,61,Black / African American,Non-Hispanic,History,601,0,0,...,1,1,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),Not Employed,NA,Never,None
100,117310,50:00.0,Female,55,Black / African American,Non-Hispanic,Office Visit,474,0,0,...,0,1,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),Full Time,"Master's degree (e.g., MA, MS, MEng, MEd, MSW,...",Never,None
134,87022,17:00.0,Female,59,White,Non-Hispanic,Telemedicine,1171,0,0,...,0,1,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),NA,NA,Quit,None
166,4420,00:00.0,Female,22,White,Non-Hispanic,Patient Outreach,1704,0,0,...,0,0,0,1572879832,R EXERCISE DAYS PER WEEK,Flowsheets Web Grid,Not Employed,11th grade,Yes,None
234,135726,23:00.0,Female,86,White,Non-Hispanic,History,1703,0,0,...,0,1,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),Retired,NA,Never,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799207,42435,27:00.0,Female,54,White,Non-Hispanic,Patient Outreach,287,0,0,...,0,1,0,1572879832,R EXERCISE DAYS PER WEEK,Flowsheet Web Navigator Section,Not Employed,NA,Yes,None
799567,187867,02:46.0,Male,62,White,Non-Hispanic,History,601,0,0,...,0,0,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),Full Time,NA,Never,None
799628,102092,42:00.0,Female,82,White,Non-Hispanic,Abstract,287,0,0,...,0,0,0,1572879832,R EXERCISE DAYS PER WEEK,Flowsheet Web Navigator Section,NA,NA,Never,None
799702,86719,02:00.0,Male,56,White,Non-Hispanic,Appointment,1140,0,0,...,0,0,0,1572879832,R EXERCISE DAYS PER WEEK,MyChart (Questionnaires),Part Time,NA,Yes,None


In [80]:
df.to_sql('MainMapped', conn, if_exists = 'replace', index = False)

800000

In [81]:
conn.commit()
conn.close()